#Kütüphaneler

In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [3]:
df = pd.read_csv("etiketli.csv")
df

,mahalle,brut_alan_m2,net_alan_m2,oda_sayisi,bina_kat_sayisi,dairenin_bulundugu_kat,bina_yas,isitma_turu,banyo_sayisi,balkon,asansor,otopark,esya_durumu,site_icinde,sahibi,aidat,depozito,fiyat
0,3,90.0,80.0,2,4.0,17,5,4,1.0,0,0,1,1,0,0,100,15000,15000.0
1,46,145.0,135.0,4,5.0,0,0,5,1.0,1,1,2,0,1,0,1500,30000,29000.0
2,17,145.0,130.0,4,4.0,7,2,4,1.0,1,0,1,0,0,1,500,25000,24750.0
3,49,230.0,200.0,4,3.0,6,10,0,2.0,0,0,1,1,0,0,0,30000,30000.0
4,47,320.0,300.0,7,8.0,12,3,4,3.0,1,1,1,0,1,0,0,37500,37500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,17,120.0,110.0,2,3.0,6,3,4,1.0,1,1,1,0,0,0,250,16000,16000.0
746,34,170.0,160.0,4,17.0,2,2,5,1.0,1,1,0,1,0,0,0,25000,25000.0
747,49,220.0,200.0,8,13.0,3,2,4,2.0,1,1,1,0,0,0,0,30000,30000.0
748,30,70.0,65.0,2,5.0,21,10,5,1.0,1,1,1,1,0,1,500,20000,20000.0


In [42]:
df.columns

Index(['mahalle', 'brut_alan_m2', 'net_alan_m2', 'oda_sayisi',
       'bina_kat_sayisi', 'dairenin_bulundugu_kat', 'bina_yas', 'isitma_turu',
       'banyo_sayisi', 'balkon', 'asansor', 'otopark', 'esya_durumu',
       'site_icinde', 'sahibi', 'aidat', 'depozito', 'fiyat'],
      dtype='object')

In [4]:
#çalışılacak sutünlar (sahibi sütunu gereksiz )
df_2 = df.copy()
df_2 = df[['mahalle', 'brut_alan_m2', 'net_alan_m2', 'oda_sayisi',
           'bina_kat_sayisi', 'dairenin_bulundugu_kat', 'bina_yas',
           'isitma_turu', 'banyo_sayisi', 'balkon', 'asansor',
           'otopark', 'esya_durumu', 'site_icinde', 'aidat', 'depozito', 'fiyat']]
df_2

,mahalle,brut_alan_m2,net_alan_m2,oda_sayisi,bina_kat_sayisi,dairenin_bulundugu_kat,bina_yas,isitma_turu,banyo_sayisi,balkon,asansor,otopark,esya_durumu,site_icinde,aidat,depozito,fiyat
0,3,90.0,80.0,2,4.0,17,5,4,1.0,0,0,1,1,0,100,15000,15000.0
1,46,145.0,135.0,4,5.0,0,0,5,1.0,1,1,2,0,1,1500,30000,29000.0
2,17,145.0,130.0,4,4.0,7,2,4,1.0,1,0,1,0,0,500,25000,24750.0
3,49,230.0,200.0,4,3.0,6,10,0,2.0,0,0,1,1,0,0,30000,30000.0
4,47,320.0,300.0,7,8.0,12,3,4,3.0,1,1,1,0,1,0,37500,37500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,17,120.0,110.0,2,3.0,6,3,4,1.0,1,1,1,0,0,250,16000,16000.0
746,34,170.0,160.0,4,17.0,2,2,5,1.0,1,1,0,1,0,0,25000,25000.0
747,49,220.0,200.0,8,13.0,3,2,4,2.0,1,1,1,0,0,0,30000,30000.0
748,30,70.0,65.0,2,5.0,21,10,5,1.0,1,1,1,1,0,500,20000,20000.0


In [5]:
# Aykırı değerlerin temizlenmesi
numerical_cols = ['brut_alan_m2', 'net_alan_m2', 'aidat', 'depozito', 'fiyat']
for col in numerical_cols:
    q1 = df_2[col].quantile(0.25)
    q3 = df_2[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df_2 = df_2[(df_2[col] >= lower_bound) & (df_2[col] <= upper_bound)]
df_2

,mahalle,brut_alan_m2,net_alan_m2,oda_sayisi,bina_kat_sayisi,dairenin_bulundugu_kat,bina_yas,isitma_turu,banyo_sayisi,balkon,asansor,otopark,esya_durumu,site_icinde,aidat,depozito,fiyat
0,3,90.0,80.0,2,4.0,17,5,4,1.0,0,0,1,1,0,100,15000,15000.0
2,17,145.0,130.0,4,4.0,7,2,4,1.0,1,0,1,0,0,500,25000,24750.0
3,49,230.0,200.0,4,3.0,6,10,0,2.0,0,0,1,1,0,0,30000,30000.0
5,23,140.0,130.0,4,10.0,16,6,4,1.0,1,1,3,1,0,0,50000,25000.0
6,45,120.0,115.0,4,3.0,0,8,4,1.0,1,0,1,0,0,100,13000,13000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,17,120.0,110.0,2,3.0,6,3,4,1.0,1,1,1,0,0,250,16000,16000.0
746,34,170.0,160.0,4,17.0,2,2,5,1.0,1,1,0,1,0,0,25000,25000.0
747,49,220.0,200.0,8,13.0,3,2,4,2.0,1,1,1,0,0,0,30000,30000.0
748,30,70.0,65.0,2,5.0,21,10,5,1.0,1,1,1,1,0,500,20000,20000.0


#**En uygun parametlerin girilmesi**

In [6]:
#bağımlı ve bağımsız değişkenler belirlendi
x = df_2.drop(['fiyat'],axis=1)
y = df_2['fiyat']
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.35,random_state = 144)

# GridSearch ile en iyi parametreleri belirleme
xgb_params = {
    "colsample_bytree": [0.6, 0.7, 0.8],  # Daha dengeli alt örnekleme oranları
    "learning_rate": [0.01, 0.05, 0.1],  # Farklı öğrenme hızları
    "max_depth": [3, 4, 5],               # Aşırı derinliği önlemek için
    "n_estimators": [50, 100, 200],
    "subsample": [0.7, 0.8, 0.9],         # Eğitim veri alt örneklemesi
    "min_child_weight": [1, 3, 5],        # Minimum veri noktası ağırlığı
    "gamma": [0, 0.1, 0.2]                # Budamayı artırmak için
}

rf_params = {
    "n_estimators": [50, 100, 200],  # Ağaç sayısı
    "max_depth": [5, 10, 15, None],   # Ağaç derinliği
    "min_samples_split": [2, 5, 10],  # Split için minimum örnek sayısı
    "min_samples_leaf": [1, 2, 4],    # Yapraklar için minimum örnek sayısı
    "bootstrap": [True, False]        # Bootstrap örnekleme
}

#**Model Eğitimi**

##XgBoost Model Eğitimi

In [7]:
#XGBoost Modeli
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=144)
grid_xg = GridSearchCV(xgb_model, xgb_params, cv=10, n_jobs=-1, verbose=2)
grid_xg.fit(x_train, y_train)
best_xgb = grid_xg.best_estimator_
print("Best XGBoost Model:", best_xgb)


Fitting 10 folds for each of 2187 candidates, totalling 21870 fits
Best XGBoost Model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=144, ...)


##Random Forest Model Eğitimi

In [10]:
# RandomForest modelinin oluşturulması
rf = RandomForestRegressor(random_state=144)
grid_rf = GridSearchCV(rf, rf_params, cv=10, n_jobs=-1, verbose=2)
grid_rf.fit(x_train, y_train)
best_rf = grid_rf.best_estimator_
print("Best Random Forest Model:", best_rf)

Fitting 10 folds for each of 216 candidates, totalling 2160 fits
Best Random Forest Model: RandomForestRegressor(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                      n_estimators=200, random_state=144)


##Voting Regresyon ve K-Çapraz Doğrulama Yöntemi

In [13]:
#iki modelin entegre çalışması için birleştirilmesi,
voting_clf = VotingRegressor(estimators=[('xgb', best_xgb), ('rf', best_rf)])

# -- K-Çapraz Doğrulama --
kf = KFold(n_splits=10, shuffle=True, random_state=144)
cv_scores = cross_val_score(voting_clf, x, y, cv=kf, scoring='r2')
print("Çapraz Doğrulama R2 Skorları:", cv_scores)
print("Ortalama R2 Skoru:", np.mean(cv_scores))


Çapraz Doğrulama R2 Skorları: [0.71083228 0.66593465 0.57581313 0.76714021 0.7416384  0.6044809
 0.63463797 0.59997903 0.52929507 0.59981957]
Ortalama R2 Skoru: 0.6429571217764426


In [14]:
#Son olarak birleştirilmiş modelin eğitimi
voting_clf.fit(x_train, y_train)

VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.6, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.1...
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=3,
                                          max_leaves=None, min_child_weight=1,
                                          missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=200,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=144, ...)),
                            ('rf',
                             RandomForestRegressor(max_depth=10,
                                                   min_samples_leaf=2,
                                                   min_samples_split=5,
                                                   n_estimators=200,
                                                   random_state=144))])

#**Tahmin ve Gerçek Değerlerin Karşılaştırılması**

In [31]:
predictions = voting_clf.predict(x_test)[10:17]
for prediction in predictions:
    print(prediction)

15702.454364592373
14855.37660191373
18492.898227780694
24095.368452944625
16581.067205143925
15759.826642410502
13153.430422342415


In [32]:
y_test[10:17]

,fiyat
80,16000.0
239,15000.0
115,12000.0
538,20000.0
640,14000.0
113,15000.0
301,13000.0


# **Model skorunu hesaplama**

In [30]:
#Model sonuçları
y_pred_train = voting_clf.predict(x_train)
y_pred_test = voting_clf.predict(x_test)
print("Eğitim Seti Skoru (R2):", r2_score(y_train, y_pred_train))
print("Test Seti Skoru (R2):", r2_score(y_test, y_pred_test))

Test_Accuracy = r2_score(y_test, y_pred_test)
# Test doğruluk oranını yüzdelik olarak hesapla
test_accuracy_percentage = Test_Accuracy * 100
print(f"Test Accuracy (Percentage): {test_accuracy_percentage:.2f}%")


Eğitim Seti Skoru (R2): 0.9235943633566213
Test Seti Skoru (R2): 0.6562556111174318
Test Accuracy (Percentage): 65.63%


#Model Veri Bilgileri

In [19]:
# XGBoost modelinin özellik önemlerini al
xgb_importance = pd.DataFrame({"Importance": best_xgb.feature_importances_},
                               index=x_train.columns)
print("XGBoost Özellik Önemi:")
print(xgb_importance)

# Random Forest modelinin özellik önemlerini al
rf_importance = pd.DataFrame({"Importance": best_rf.feature_importances_},
                              index=x_train.columns)
print("Random Forest Özellik Önemi:")
print(rf_importance)


XGBoost Özellik Önemi:
                        Importance
mahalle                   0.047224
brut_alan_m2              0.056337
net_alan_m2               0.078454
oda_sayisi                0.086915
bina_kat_sayisi           0.052144
dairenin_bulundugu_kat    0.046202
bina_yas                  0.021530
isitma_turu               0.027935
banyo_sayisi              0.145800
balkon                    0.050209
asansor                   0.029740
otopark                   0.029452
esya_durumu               0.137190
site_icinde               0.042735
aidat                     0.024083
depozito                  0.124051
Random Forest Özellik Önemi:
                        Importance
mahalle                   0.082164
brut_alan_m2              0.221141
net_alan_m2               0.085991
oda_sayisi                0.021750
bina_kat_sayisi           0.046350
dairenin_bulundugu_kat    0.058474
bina_yas                  0.026627
isitma_turu               0.009983
banyo_sayisi              0.028073
bal

In [21]:
print(x_train.head())
print(y_train.head())

     mahalle  brut_alan_m2  net_alan_m2  oda_sayisi  bina_kat_sayisi  \
645       47          62.0         52.0           0              3.0   
406        0         125.0        115.0           2              4.0   
174       40         150.0        140.0           4             11.0   
695       29         150.0        140.0           4              4.0   
295       45         145.0        135.0           4              4.0   

     dairenin_bulundugu_kat  bina_yas  isitma_turu  banyo_sayisi  balkon  \
645                       6        10            4           1.0       0   
406                       7         5            4           1.0       1   
174                       7         3            4           2.0       1   
695                       0         3           11           2.0       1   
295                       6         6           11           1.0       1   

     asansor  otopark  esya_durumu  site_icinde  aidat  depozito  
645        0        1            1         

In [22]:
print(x_test.head())
print(y_test.head())

     mahalle  brut_alan_m2  net_alan_m2  oda_sayisi  bina_kat_sayisi  \
372       17          85.0         75.0           2              2.0   
41        46          85.0         70.0           2              5.0   
114       49         175.0        165.0           4              4.0   
105       26         130.0        125.0           4              4.0   
606       22         140.0        135.0           4              4.0   

     dairenin_bulundugu_kat  bina_yas  isitma_turu  banyo_sayisi  balkon  \
372                      20         4            5           1.0       1   
41                        0         2            4           1.0       0   
114                       7         5            5           1.0       1   
105                      17         2            4           1.0       1   
606                       7         2            5           2.0       1   

     asansor  otopark  esya_durumu  site_icinde  aidat  depozito  
372        0        3            0         